In [1]:
## Get the library Setup
from equity_downloader import *
from datetime import date, timedelta, datetime
import random
import time
import random
import requests

import pandas as pd
import numpy as np
import xgboost as xgb
from itertools import chain
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score
import os
import re
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### Import the index and base equity file

In [2]:
index_base = pd.read_csv("Misc Files/index_base.csv", low_memory=False)
index_base['Index Date'] = [convert_date(x, format= "%d-%m-%Y") for x in index_base['Index Date'].tolist()]
index_base['DATE'] = pd.to_datetime(index_base['Index Date'], format='%Y-%m-%d')
index_grp = index_base.groupby('Index Name')['DATE'].count().reset_index()
index_grp = index_grp[index_grp['DATE']>800]
index_base = index_base[index_base['Index Name'].isin(index_grp['Index Name'].tolist())]
index_base.drop_duplicates(['Index Name', 'DATE'], inplace=True)

In [3]:
eq_df_industry = pd.read_csv("Misc Files/bhavcopy_modeldf.csv", low_memory=False)
eq_df_industry['DATE'] = pd.to_datetime(eq_df_industry['DATE'], format='%Y-%m-%d')
eq_df_industry.drop_duplicates(['SYMBOL', 'DATE'], inplace=True)

In [4]:
# Apply the filter
SYMBOL_COLS = eq_df_industry['SYMBOL'].unique()
industry_index = ['Nifty 50', 'Nifty Auto','Nifty Bank','Nifty Energy','Nifty Financial Services','Nifty FMCG'
                  ,'Nifty IT','Nifty Media','Nifty Metal','Nifty MNC','Nifty Pharma','Nifty PSU Bank','Nifty Realty'
                  ,'Nifty India Consumption','Nifty Commodities','Nifty Dividend Opportunities 50','Nifty Infrastructure'
                  ,'Nifty PSE','Nifty Services Sector','Nifty India Digital','Nifty Mobility','Nifty India Defence'
                  ,'Nifty Financial Services Ex-Bank','Nifty Housing','Nifty Transportation & Logistics'
                  ,'Nifty MidSmall Financial Services','Nifty MidSmall Healthcare','Nifty MidSmall IT & Telecom'
                  ,'Nifty MidSmall India Consumption','Nifty REITs & InvITs','Nifty Core Housing','Nifty Consumer Durables'
                  ,'Nifty Non-Cyclical Consumer','Nifty India Manufacturing','Nifty Private Bank','NIFTY SME EMERGE'
                  ,'Nifty Oil & Gas','Nifty Healthcare Index']


In [ ]:
corr_drop=False
perf_agg_14 = pd.DataFrame(columns = ['SYMBOL', 'Best_Cutoff_14', 'Rank_Cutoff_14','Train_Accuracy_14', 'Test_Accuracy_14', 'AUC_TRAIN_14', 'AUC_test_14'])
perf_agg_30 = pd.DataFrame(columns = ['SYMBOL', 'Best_Cutoff_30', 'Rank_Cutoff_30', 'Train_Accuracy_30', 'Test_Accuracy_30', 'AUC_TRAIN_30', 'AUC_test_30'])

target_df = {'TARGET_14':perf_agg_14, 'TARGET_30':perf_agg_30}
target_col = ['TARGET_14', 'TARGET_30']
target_suffix = {'TARGET_14':['Prob_14', 'Predicted_14'], 'TARGET_30':['Prob_30', 'Predicted_30']}


for i in tqdm(range(len(SYMBOL_COLS))):
    SYMBOL = SYMBOL_COLS[i]
    #----------------------------------Include index features----------------------------------------------
    eq_symbol_in = eq_data_pred(eq_df_industry = eq_df_industry, symbol=SYMBOL, keep_ma=True)
    eq_symbol_index = pd.merge(eq_symbol_in[['DATE', 'CLOSE_PRICE']], index_base, on=["DATE"])

    tt = eq_symbol_index[['Index Name', 'CLOSE_PRICE', 'Closing Index Value']].groupby('Index Name').corr().reset_index()
    tt1 = tt[tt['level_1']=='CLOSE_PRICE'][['Index Name', 'Closing Index Value']].copy()
    tt2 = tt1[(tt1['Closing Index Value']>0.8)].copy()
    if tt2.shape[0]>0:
        index = tt2.sort_values(by="Closing Index Value").tail(1)['Index Name'].tolist()[0]
    else:
        index=None

    if index is not None:
        index_select=eq_symbol_index[eq_symbol_index['Index Name']==index][['Index Name', 'DATE', 'Closing Index Value']].copy()
        index_select.sort_values(by = "DATE", inplace=True)
        # There might be few empty values where index value is missing. First get the full dates
        index_full_date = pd.merge(eq_symbol_in[['DATE']], index_select, on=["DATE"], how="left")
        index_full_date['Closing Index Value'] = index_full_date['Closing Index Value'].ffill()
        index_full_date['GRAD_INDEX'] = np.gradient(index_full_date['Closing Index Value'], np.arange(index_full_date['Closing Index Value'].shape[0]))
        lags = [7, 14, 30, 45, 60, 90, 180]
        for lag in lags:
            grad_col = f'GRAD_INDEX_{lag}'
            index_full_date['temp'] = index_full_date['Closing Index Value'].rolling(lag).mean()
            index_full_date[grad_col] = np.gradient(index_full_date['temp'], np.arange(index_full_date['temp'].shape[0]))

        index_full_date.drop(['Index Name','temp'],axis=1,inplace=True)

        eq_symbol  = pd.merge(eq_symbol_in, index_full_date, on='DATE', how="left")

    else:
        eq_symbol = eq_symbol_in.copy()

    #-----------------------------------------------------------------------------------------------------------
    pred_cols = eq_symbol.columns[eq_symbol.columns.str.contains('MA_')|eq_symbol.columns.str.contains('CROSSOVER')|eq_symbol.columns.str.contains('GRAD')]
    eq_symbol_f=eq_symbol[list(chain.from_iterable([['DATE','DAY', 'CLOSE_PRICE'], target_col,list(pred_cols)]))].dropna()
    ignore_ma = ['MA_7', 'MA_14','MA_30', 'MA_45', 'MA_60', 'MA_90', 'MA_180']
    pred_cols = list(chain.from_iterable([[x for x in pred_cols if x not in ignore_ma], ['DAY']]))
    
    #--------------------------------Drop Strongly correlated variables------------------------------------------------------

    if corr_drop:
        pred_cols_f = [x for x in pred_cols if x not in drop_corr_vars(df = eq_symbol_f, cols = pred_cols, cutoff=0.8)]
    else:
        pred_cols_f = pred_cols.copy()
    
    eq_symbol_last6=eq_symbol[eq_symbol['DATE']>=(datetime.today()-timedelta(days=180))][list(chain.from_iterable([['SYMBOL', 'DATE', 'CLOSE_PRICE']
                                                                                                                   , target_col,list(pred_cols),ignore_ma]))].copy()

    #---------------------------Prepare for the model------------------------------------
    for k, target in enumerate(target_col):
        train_mat_x, test_mat_x, train_mat_y, test_mat_y = train_test_split(eq_symbol_f[pred_cols_f], eq_symbol_f[target], test_size=0.3, random_state=42)

        dtrain = xgb.DMatrix(train_mat_x, label=train_mat_y.values.reshape(1,-1))
        dtest = xgb.DMatrix(test_mat_x, label=test_mat_y.values.reshape(1,-1))
        data_all = xgb.DMatrix(eq_symbol_last6[pred_cols_f])
        
        params = { 'objective': 'binary:logistic','eval_metric': 'auc', 'gamma':1, 'max_depth':5
                                          , 'reg_lambda':1,'reg_alpha':1,  'learning_rate':0.1, 'min_child_weight':10}
        # Train the classifier
        model = xgb.train(params, dtrain,100)

        # Make predictions on the test set
        test_pred = model.predict(dtest)
        train_pred = model.predict(dtrain)
        all_pred = model.predict(data_all)
        
        # Compute the AUC
        auc_test = roc_auc_score(test_mat_y, test_pred)
        auc_train = roc_auc_score(train_mat_y, train_pred)
        row={'SYMBOL':SYMBOL,'AUC_TRAIN':auc_train,'AUC_TEST':auc_test}

        model_perf = best_cutoff_acc(train_actual = train_mat_y, train_prediction=train_pred, test_actual = test_mat_y, test_prediction = test_pred)

        model_perf['SYMBOL']=SYMBOL
        model_perf['AUC_TRAIN']=np.round(100*auc_train,1)
        model_perf['AUC_test']=np.round(100*auc_test,1)
    
        target_df[target].loc[i] = [model_perf['SYMBOL'], model_perf['Best_Cutoff'], model_perf['Rank_Cutoff'], model_perf['Train_Accuracy']
                           , model_perf['Test_Accuracy'], model_perf['AUC_TRAIN'], model_perf['AUC_test'] ]
        eq_symbol_last6[target_suffix[target][0]] = all_pred
        eq_symbol_last6[target_suffix[target][1]] = np.where(all_pred>=model_perf['Best_Cutoff'], 1,0)

        
    if (i==0):
        pred_last6 = eq_symbol_last6.copy()
    else:
        pred_last6 = pd.concat([pred_last6, eq_symbol_last6],ignore_index=True)

 86%|████████████████████████████████████████████████████████████████████▉           | 526/610 [04:26<00:43,  1.92it/s]

### Transform and Save

In [ ]:
keep_cols = ['SYMBOL', 'DATE', 'CLOSE_PRICE', 'Prob_14', 'Prob_30', 'Predicted_14', 'Predicted_30'
             , 'CROSSOVER_14', 'CROSSOVER_30', 'CROSSOVER_45', 'CROSSOVER_60', 'CROSSOVER_90', 'CROSSOVER_180']

final_col_order = ['SYMBOL', 'COMPANY NAME', 'INDUSTRY', 'MCAP_INLAKHS', 'DATE', 'CLOSE_PRICE', 'Top_Rank', 'Rank', 'Score'
                   , 'Prob_14', 'Prob_30', 'Predicted_14', 'Predicted_30', 'Test_Accuracy_14', 'Test_Accuracy_30'
                   , 'Pred14_0_TGT_0', 'Pred14_0_TGT_1', 'Pred14_1_TGT_0', 'Pred14_1_TGT_1', 'Pred14_0_Acc', 'Pred14_1_Acc'
                   , 'Target14_0_Sum', 'Target14_1_Sum', 'Pred30_0_TGT_0', 'Pred30_0_TGT_1', 'Pred30_1_TGT_0', 'Pred30_1_TGT_1'
                   , 'Pred30_0_Acc', 'Pred30_1_Acc', 'Target30_0_Sum', 'Target30_1_Sum'
                   , 'AUC_test_14', 'AUC_test_30' , 'Best_Cutoff_14', 'Rank_Cutoff_14', 'Best_Cutoff_30', 'Rank_Cutoff_30'
                   , 'CROSSOVER_14', 'CROSSOVER_30', 'CROSSOVER_45', 'CROSSOVER_60', 'CROSSOVER_90', 'CROSSOVER_180'
                   , 'Train_Accuracy_14','Train_Accuracy_30', 'AUC_TRAIN_14','AUC_TRAIN_30']

sec_mcap = pd.read_csv("E:/Equity Research/Misc Files/Eq_Mcap_Industry_data.csv")
# Get model metrics for 30 days target
perf_last6_30 = pred_last6.pivot_table(index='SYMBOL', columns = ['Predicted_30', 'TARGET_30'], aggfunc = 'count', values = 'CLOSE_PRICE', fill_value=0).reset_index()
perf_last6_30.columns = ['SYMBOL', 'Pred30_0_TGT_0', 'Pred30_0_TGT_1', 'Pred30_1_TGT_0', 'Pred30_1_TGT_1']
perf_last6_30['Pred30_0_Acc'] = np.round(100*perf_last6_30['Pred30_0_TGT_0']/(perf_last6_30['Pred30_0_TGT_0']+perf_last6_30['Pred30_0_TGT_1']),1)
perf_last6_30['Pred30_1_Acc'] = np.round(100*perf_last6_30['Pred30_1_TGT_1']/(perf_last6_30['Pred30_1_TGT_0']+perf_last6_30['Pred30_1_TGT_1']),1)
perf_last6_30['Target30_0_Sum'] = perf_last6_30['Pred30_0_TGT_0']+perf_last6_30['Pred30_1_TGT_0']
perf_last6_30['Target30_1_Sum'] = perf_last6_30['Pred30_0_TGT_1']+perf_last6_30['Pred30_1_TGT_1']

# Get model metrics for 14 days target
perf_last6_14 = pred_last6.pivot_table(index='SYMBOL', columns = ['Predicted_14', 'TARGET_14'], aggfunc = 'count', values = 'CLOSE_PRICE', fill_value=0).reset_index()
perf_last6_14.columns = ['SYMBOL', 'Pred14_0_TGT_0', 'Pred14_0_TGT_1', 'Pred14_1_TGT_0', 'Pred14_1_TGT_1']
perf_last6_14['Pred14_0_Acc'] = np.round(100*perf_last6_14['Pred14_0_TGT_0']/(perf_last6_14['Pred14_0_TGT_0']+perf_last6_14['Pred14_0_TGT_1']),1)
perf_last6_14['Pred14_1_Acc'] = np.round(100*perf_last6_14['Pred14_1_TGT_1']/(perf_last6_14['Pred14_1_TGT_0']+perf_last6_14['Pred14_1_TGT_1']),1)
perf_last6_14['Target14_0_Sum'] = perf_last6_14['Pred14_0_TGT_0']+perf_last6_14['Pred14_1_TGT_0']
perf_last6_14['Target14_1_Sum'] = perf_last6_14['Pred14_0_TGT_1']+perf_last6_14['Pred14_1_TGT_1']

pred_latest_df = pred_last6[pred_last6['DATE']==pred_last6['DATE'].max()][keep_cols].copy()

pred_latest_df = pd.merge(sec_mcap, pred_latest_df, on='SYMBOL')

model_perf_df = pd.merge(perf_agg_14, perf_agg_30, on='SYMBOL')

model_perf_df = pd.merge(pred_latest_df, model_perf_df, on ="SYMBOL")

model_perf_df = pd.merge(model_perf_df, perf_last6_14, on ="SYMBOL")

model_perf_df = pd.merge(model_perf_df, perf_last6_30, on ="SYMBOL")

model_perf_df['Pred14_1_Acc'] = model_perf_df['Pred14_1_Acc'].fillna(0)
model_perf_df['Pred30_1_Acc'] = model_perf_df['Pred30_1_Acc'].fillna(0)

C1 = 100*(model_perf_df['Prob_14']*model_perf_df['Predicted_14']+model_perf_df['Prob_30']*model_perf_df['Predicted_30']) # Joint Probability
C2 = (0.5*model_perf_df['Test_Accuracy_14'] + 0.5*model_perf_df['Pred14_1_Acc']) # Recent vs Overall Test Accuracy
C3 = (0.5*model_perf_df['Test_Accuracy_30'] + 0.5*model_perf_df['Pred30_1_Acc']) # Recent vs Overall Test Accuracy
C4 = 100*(0.2*model_perf_df['CROSSOVER_14']+ 0.2*model_perf_df['CROSSOVER_30'] + 0.2*model_perf_df['CROSSOVER_45'] + 0.2*model_perf_df['CROSSOVER_60'] + 0.1*model_perf_df['CROSSOVER_90'] + 0.1*model_perf_df['CROSSOVER_180'])

model_perf_df['Score'] = C1+C2+C3+C4
model_perf_df['Rank'] = model_perf_df['Score'].rank(method='dense', ascending=False).astype(int)

model_perf_df['Top_Rank'] = np.where(model_perf_df['Rank']<=10, model_perf_df['Rank'].astype(str), "10+")

model_perf_df = model_perf_df[final_col_order]

file_date = pred_last6['DATE'].max().date()

pred_last6['Pred_Acc_14'] = np.where(pred_last6['TARGET_14'].isna(), np.NaN
                                     , np.where(pred_last6['TARGET_14']==pred_last6['Predicted_14'],1,0))

pred_last6['Predicted_14'] = np.where(pred_last6['TARGET_14'].isna(), pred_last6['Predicted_14'], np.NaN )


pred_last6['Pred_Acc_30'] = np.where(pred_last6['TARGET_30'].isna(), np.NaN
                                     , np.where(pred_last6['TARGET_30']==pred_last6['Predicted_30'],1,0))

pred_last6['Predicted_30'] = np.where(pred_last6['TARGET_30'].isna(), pred_last6['Predicted_30'], np.NaN )

pred_last6['Yr_Month'] = [f'{x[0:3]},{y-2000}' for x,y in zip(pred_last6['DATE'].dt.month_name().tolist(), pred_last6['DATE'].dt.year.tolist())]

# Add the rank to pred last 6
pred_last6 = pd.merge(model_perf_df[['SYMBOL', 'Top_Rank', 'Rank']], pred_last6, on='SYMBOL')

model_perf_df.to_csv(f"E:/Equity Research/Model_Predictions/model_perf_df_{file_date}.csv", index=False)
pred_last6.to_csv(f"E:/Equity Research/Model_Predictions/pred_last6_{file_date}.csv", index=False)

model_perf_df.to_csv(f"E:/Equity Research/Model_Predictions/model_perf_df.csv", index=False)
pred_last6.to_csv(f"E:/Equity Research/Model_Predictions/pred_last6.csv", index=False)


### Summary Statistics

In [ ]:
model_perf_df[['Predicted_14', 'Predicted_30']].value_counts().reset_index()

In [ ]:
model_perf_df[[ 'Test_Accuracy_14', 'Test_Accuracy_30', 'AUC_test_14', 'AUC_test_30']].describe()

In [ ]:
model_perf_df[[ 'Train_Accuracy_14', 'Train_Accuracy_30', 'AUC_TRAIN_14', 'AUC_TRAIN_30']].describe()

In [ ]:
# pd.set_option('display.max_columns', None)
model_perf_df[[ 'Best_Cutoff_14', 'Rank_Cutoff_14', 'Best_Cutoff_30', 'Rank_Cutoff_30']].describe()

In [ ]:
# pd.set_option('display.max_columns', None)
# model_perf_df.head(10)